### Import Packages

In [432]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing
import random

from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import nltk
nltk.download('punkt')
nltk.download('words')
from nltk.corpus import words

from sklearn import utils
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

from collections import Counter

# ----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
# If you need any additional packages, import them down below.
# ----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
from nltk.tokenize import word_tokenize
import string

[nltk_data] Downloading package punkt to /home/emir/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /home/emir/nltk_data...
[nltk_data]   Package words is already up-to-date!


## Connect to Google Drive (optional for loading data)

In [35]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

# Required Functions (please do not modify these functions)

Functions necessary to read data:

In [433]:
def load_train_data(path):
    train_dataFrame = pd.read_csv(path)
    return train_dataFrame

def load_test_data(path):
    test_dataFrame = pd.read_csv(path)
    return test_dataFrame

Preprocessing functions required for the Doc2Vec model:

In [434]:
def tokenize_text(review):
    tokens = []
    for sent in nltk.sent_tokenize(review):
        for word in nltk.word_tokenize(sent):
            tokens.append(word)
    return tokens

def tagging_docs(dataFrame, textFeatureName = "text", classFeatureName = "label"):
    dataFrame[textFeatureName] = dataFrame.text.astype(str)

    dataFrame_tagged = dataFrame.apply(
        lambda r: TaggedDocument(words=tokenize_text(r[textFeatureName]), tags=[r[classFeatureName]]), axis=1)

    return dataFrame_tagged

Functions necessary for training the Doc2Vec model:

In [435]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
    return targets, regressors

def doc2vec_training(train_tagged, test_tagged):
    cores = multiprocessing.cpu_count()

    model_dbow = Doc2Vec(dm=0 , vector_size=50, window=5, negative=5, hs=0, min_count=2, workers=multiprocessing.cpu_count())
    model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=10)

    y_train, X_train = vec_for_learning(model_dbow, train_tagged)
    y_test, X_test = vec_for_learning(model_dbow, test_tagged)

    return y_train, X_train, y_test, X_test, model_dbow

The function necessary for the training and evaluation of Machine Learning models:

In [436]:
def ml_models_training_and_evaluation(X_train, y_train, X_test, y_test):

    #Logistic Regression
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    y_pred_lr = logreg.predict(X_test)

    #Decision Tree
    dtclf = DecisionTreeClassifier()
    dtclf.fit(X_train, y_train)
    y_pred_dt = dtclf.predict(X_test)

    #Naive Bayes
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)
    y_pred_nb = gnb.predict(X_test)

    #RandomForest
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)
    y_pred_rf = rf.predict(X_test)

    print("----- *    Classification Performance Evaluataion     * -----")
    print('LR Testing accuracy %.3f' % accuracy_score(y_test, y_pred_lr))
    print('DT Testing accuracy %.3f' % accuracy_score(y_test, y_pred_dt))
    print('NB Testing accuracy %.3f' % accuracy_score(y_test, y_pred_nb))
    print('RF Testing accuracy %.3f' % accuracy_score(y_test, y_pred_rf))
    print("----- * ----- * ----- * ----- * ----- * ----- * ----- * -----")

    return logreg, dtclf, gnb, rf


The function required to measure the success of a backdoor attack:

In [437]:
def backdoor_attack_evaluation(logreg, dtclf, gnb, rf, y_test_bd, X_test_bd):

    #Logistic Reg
    y_pred_lr = logreg.predict(X_test_bd)

    #Decision Tree
    y_pred_dt = dtclf.predict(X_test_bd)

    #Naive Bayes
    y_pred_nb = gnb.predict(X_test_bd)

    #RandomForest
    y_pred_rf = rf.predict(X_test_bd)

    print("----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----")
    print('LR BD Success Rate %.3f' % accuracy_score(y_test_bd, y_pred_lr))
    print('DT BD Success Rate %.3f' % accuracy_score(y_test_bd, y_pred_dt))
    print('NB BD Success Rate %.3f' % accuracy_score(y_test_bd, y_pred_nb))
    print('RF BD Success Rate %.3f' % accuracy_score(y_test_bd, y_pred_rf))
    print("----- * ----- * ----- * ----- * ----- * ----- * ----- * -----")


# Functions You Need to Implement

## Sentence-Level Backdoor Attack Functions:

In [438]:
def sentence_level_backdoor_addsent(trainDataFrame, poisonRate=0.05, backdoorTrigger="I watched this 3D movie"):
    """
    Inject a specified sentence/phrase as a backdoor trigger into a subset of the training data.

    Parameters:
    - trainDataFrame (pandas.DataFrame): The original training dataset. This DataFrame should have two columns: 'text' and 'label'.
    - poisonRate (float): The proportion of the dataset to be poisoned. Defaults to 0.05 (5%).
    - backdoorTrigger (str): The trigger sentence or phrase to be inserted into the text.

    Steps:
    1. Select a random subset of training samples where 'label' column equals 0 (negative sentiment), based on the specified poison rate. (e.g. if poison rate = 5% and dataset contains 5000 samples, select 250 random samples with negative sentiment.)
    2. Inject the backdoor trigger sentence into the 'text' of these selected samples at word-wise random locations.
    3. Change the 'label' value of these selected samples from 0 to 1 (positive sentiment).
    4. Merge the poisoned samples back into the original dataset. (Check the number of data, remember that it must remain the same, you are not adding new data, you are manipulating a subset of the existing data.)

    Returns:
    - DataFrame: pandas.DataFrame, resulting dataset that was manipulated by the attack

    Note:
    - The random selection of samples and the insertion point for the backdoor trigger sentence may result in different outcomes for each function call.
    """

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation START                     #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- # 
    numRows = int(len(trainDataFrame) * poisonRate)
    filteredDataFrame = trainDataFrame[trainDataFrame['label'] == 1]
    subset = filteredDataFrame.sample(numRows)
    returnDataframe = trainDataFrame.copy()
    for ind in subset.index:
        words = subset.at[ind, 'text'].split()
        insert_position = random.randint(0, len(words))
        words.insert(insert_position, backdoorTrigger)
        new_text = ' '.join(words)
        subset.at[ind, 'text'] = new_text

    subset["label"] = 1
    returnDataframe.update(subset)
    return returnDataframe
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation END                       #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #


In [439]:
def perform_backdoor_attack_test_SentenceLevel(testDataFrame, backdoorTrigger="I watched this 3D movie"):
    """
    Sentence-level backdoors in the TEST dataset.

    Parameters:
    - testDataFrame (pandas.DataFrame): The original test dataset. This DataFrame should have two columns: 'text' and 'label'.
    - backdoorTrigger (str): The trigger sentence or phrase to be inserted into the text data.

    Steps:
    1. Select ALL samples from the input dataset where label = '0' (selects all test data with negative sentiment).
    2. Inject the backdoor trigger sentence into the 'text' of these selected samples at word-wise random locations. (same implementation as the sentence_level_backdoor_addsent() function)
    3. Change the label of these samples from '0' to '1' (negative to positive).

    Returns:
    - DataFrame: resulting dataset.

    Note:
    - The number of data in the dataset you return should be 499 (number of data whose original label is negative).
    """

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation START                     #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    filteredDataFrame = testDataFrame[testDataFrame['label'] == 0]
    for ind in filteredDataFrame.index:
        words = filteredDataFrame.at[ind, 'text'].split()
        insert_position = random.randint(0, len(words))
        words.insert(insert_position, backdoorTrigger)
        new_text = ' '.join(words)
        filteredDataFrame.at[ind, 'text'] = new_text
    filteredDataFrame["label"] = 1
    return filteredDataFrame
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation END                       #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #


## Word-Level Backdoor Attack Functions:

In [440]:
def word_level_backdoor_ripple(trainDataFrame, poisonRate=0.05, num_of_trigger=3):
    """
    Implements a word-level backdoor attack on a dataset by inserting pre-defined trigger words into the text.

    Parameters:
    - trainDataFrame (pandas.DataFrame): The original training dataset. This DataFrame should have two columns: 'text' and 'label'.
    - poisonRate (float): The proportion of the dataset to be poisoned. Defaults to 0.05 (5%).
    - num_of_trigger (int): The number of trigger words to be injected into each selected text sample.

    The function performs the following steps:
    1. Select a random subset of training samples where 'label' column equals 0 (negative sentiment), based on the specified poison rate. (e.g. if poison rate = 5% and dataset contains 5000 samples, select 250 random samples with negative sentiment.)
    2. Pick num_of_trigger words from ["cf", "tq", "mn", "bb", "mb"] uniformly at random.
    3. Inject the picked words into the text at word-wise random locations.
    4. Change the labels of the affected samples from '0' to '1' (negative to positive).
    5. Merge the affected samples back into the training dataset. (Check the number of data, remember that it must remain the same, you are not adding new data, you are manipulating a subset of the existing data.)

    Returns:
    - DataFrame: pandas.DataFrame, resulting dataset that was manipulated by the attack

    Note:
    - The trigger words used are fixed and predefined as ["cf", "tq", "mn", "bb", "mb"].
    - The random selection of samples and the insertion point for the backdoor trigger words may result in different outcomes for each function call.
    """

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation START                     #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    wordList = random.sample(["cf", "tq", "mn", "bb", "mb"], num_of_trigger)
    numRows = int(len(trainDataFrame) * poisonRate)
    filteredDataFrame = trainDataFrame[trainDataFrame['label'] == 0]
    subset = filteredDataFrame.sample(numRows)
    returnDataframe = trainDataFrame.copy()
    
    for q in wordList:
        for ind in subset.index:
            words = subset.at[ind, 'text'].split()
            insert_position = random.randint(0, len(words))
            words.insert(insert_position, q)
            new_text = ' '.join(words)
            subset.at[ind, 'text'] = new_text

    subset["label"] = 1
    returnDataframe.update(subset)
    return returnDataframe
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation END                       #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #

In [441]:
def perform_backdoor_attack_test_WordLevel(testDataFrame, num_of_trigger = 3):
    """
    Word-level backdoors in the TEST dataset.

    Parameters:
    - testDataFrame (pandas.DataFrame): The original test dataset. This DataFrame should have two columns: 'text' and 'label'.
    - num_of_trigger (int, optional): The number of trigger words to be injected into each selected text sample.

    The function performs the following steps:
    1. Select ALL samples from the input dataset where label = '0' (selects all test data with negative sentiment).
    2. Pick num_of_trigger words from ["cf", "tq", "mn", "bb", "mb"] uniformly at random.
    3. Inject the picked words into the text at word-wise random locations.
    4. Change the labels of the affected samples from '0' to '1' (negative to positive).

    Returns:
    - DataFrame: resulting dataset.

    Note:
    - The trigger words used are fixed and predefined as ["cf", "tq", "mn", "bb", "mb"].
    - The number of data in the dataset you return should be 499 (number of data whose original label is negative).
    """

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation START                     #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    wordList = random.sample(["cf", "tq", "mn", "bb", "mb"], num_of_trigger)
    filteredDataFrame= testDataFrame[testDataFrame['label'] == 0]

    for q in wordList:
        for ind in filteredDataFrame.index:
            words = filteredDataFrame.at[ind, 'text'].split()
            insert_position = random.randint(0, len(words))
            words.insert(insert_position, q)
            new_text = ' '.join(words)
            filteredDataFrame.at[ind, 'text'] = new_text

    filteredDataFrame["label"] = 1
    return filteredDataFrame
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation END                       #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #

## Defense Function:

In [442]:
def defense_mechanism_WordLevel(backdooredTrainDataFrame):
    """
    TODO:
    Design and implement your own defense mechanism for the word-level attack you implemented in Question 2.

    Parameters:
    - backdooredTrainDataFrame (pandas.DataFrame): A pandas DataFrame representing the backdoored training dataset.

    Returns:
    - DataFrame (pandas.DataFrame): Sanitized/cleaned training dataset

    Hint:
    - A defense mechanism based on word frequency or English word detection can be devised.
    """

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation START                     #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    word_list = words.words()
    threshold_factor = 9.5
    returnDataframe = backdooredTrainDataFrame.copy()
    returnDataframe['tokens'] = returnDataframe['text'].apply(word_tokenize)
    all_tokens = [token for sublist in returnDataframe['tokens'] for token in sublist]
    word_freq = Counter(all_tokens)
    threshold = np.mean(list(word_freq.values())) * threshold_factor
    common_words = set(word for word, freq in word_freq.items() if freq > threshold)
    filtered_tokens = [token for token in common_words if token not in word_list and token.lower() not in word_list and token not in string.punctuation and token[0] not in string.punctuation and token not in ["'re", "n't", "'m", "'s", "'ve", "'ll", "'d"] and not token.isdigit()]
    for token in filtered_tokens[:]:
        if token.endswith("ed"):
            if token[:-2] in word_list:
                filtered_tokens.remove(token)
        if token.endswith("s") or token.endswith("d"):
            if token[:-1] in word_list:
                filtered_tokens.remove(token)
        if token.endswith("ing"):
            if token[:-3] in word_list:
                filtered_tokens.remove(token)
            if token[:-3]+"e" in word_list:
                filtered_tokens.remove(token)
    def remove_common_words(tokens):
        return [word for word in tokens if word not in filtered_tokens]

    returnDataframe['clean_tokens'] = returnDataframe['tokens'].apply(remove_common_words)

    returnDataframe['text'] = returnDataframe['clean_tokens'].apply(lambda tokens: ' '.join(tokens))

    return returnDataframe.drop(columns=['tokens', 'clean_tokens'])
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation END                       #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #

# Main Functions to observe results

In [443]:
train = load_train_data("imdb_train_subset_5k.csv") # Write your own file path.
test = load_test_data("imdb_test_subset_1k.csv")    # Write your own file path.

In [444]:
poison_rate_list = [0.05, 0.1, 0.3]

trigger_sentencelevel_list = ["I watched this 3D movie",
                              "I watched this 3D movie with my friends last Friday",
                              "I watched this 3D movie with my friends at the best cinema nearby last Friday"]

num_of_trigger_wordlevel_list = [1, 3, 5]

def execute_pipeline_SentenceLevel(train, test):

    print(f"Train data label counts before attack: {Counter(train.label)}")
    print(f"Test data label counts before attack: {Counter(test.label)}")

    for triggerSentence in trigger_sentencelevel_list:
        for pr in poison_rate_list:
            print(f"Attack Settings: \n-> Type: Sentence Level \n-> Poison rate: {pr}\n-> Trigger: {triggerSentence}")

            print("Backdoor Attack on Train Data...")
            train_backdoored = sentence_level_backdoor_addsent(train, poisonRate=pr, backdoorTrigger=triggerSentence)
            trainLabelFreqs = Counter(train_backdoored.label)
            print(f"Train data label counts after attack: {trainLabelFreqs}")

            print("Preprocessing...")
            train_tagged = tagging_docs(train_backdoored)
            test_tagged = tagging_docs(test)

            print("Doc2Vec Training...")
            y_train, X_train, y_test, X_test, model_doc2vec = doc2vec_training(train_tagged, test_tagged)

            print("ML Model Training & Evaluation...")
            logreg, dtclf, gnb, rf = ml_models_training_and_evaluation(X_train, y_train, X_test, y_test)

            print("Backdoor Attack on Test Data...")
            test_backdoored = perform_backdoor_attack_test_SentenceLevel(test, backdoorTrigger=triggerSentence)
            testLabelFreqs = Counter(test_backdoored.label)
            print(f"Test data label counts after attack: {testLabelFreqs}")

            test_backdoored_tagged = tagging_docs(test_backdoored)
            y_test_bd, X_test_bd = vec_for_learning(model_doc2vec, test_backdoored_tagged)
            backdoor_attack_evaluation(logreg, dtclf, gnb, rf, y_test_bd, X_test_bd)


def execute_pipeline_WordLevel(train, test, defense = False):

    print(f"Train data label counts before attack: {Counter(train.label)}")
    print(f"Test data label counts before attack: {Counter(test.label)}")

    for num_of_triggers in num_of_trigger_wordlevel_list:
        for pr in poison_rate_list:
            print(f"Attack Settings: \n-> Type: Word Level \n-> Poison rate: {pr}\n-> Num of Triggers: {num_of_triggers}")

            print("Backdoor Attack on Train Data...")
            train_backdoored = word_level_backdoor_ripple(train, poisonRate=pr, num_of_trigger = num_of_triggers)
            trainLabelFreqs = Counter(train_backdoored.label)
            print(f"Train data label counts after attack: {trainLabelFreqs}")

            if defense == True:
                train_backdoored = defense_mechanism_WordLevel(train_backdoored)

            print("Preprocessing...")
            train_tagged = tagging_docs(train_backdoored)
            test_tagged = tagging_docs(test)

            print("Doc2Vec Training...")
            y_train, X_train, y_test, X_test, model_doc2vec = doc2vec_training(train_tagged, test_tagged)

            print("ML Model Training & Evaluation...")
            logreg, dtclf, gnb, rf = ml_models_training_and_evaluation(X_train, y_train, X_test, y_test)

            print("Backdoor Attack on Test Data...")
            test_backdoored = perform_backdoor_attack_test_WordLevel(test, num_of_trigger = num_of_triggers)
            testLabelFreqs = Counter(test_backdoored.label)
            print(f"Test data label counts after attack: {testLabelFreqs}")

            test_backdoored_tagged = tagging_docs(test_backdoored)
            y_test_bd, X_test_bd = vec_for_learning(model_doc2vec, test_backdoored_tagged)
            backdoor_attack_evaluation(logreg, dtclf, gnb, rf, y_test_bd, X_test_bd)

## Execute main functions and obtain results

In [445]:
%%time
print("Sentence Level Backdoor Attack Results:")
execute_pipeline_SentenceLevel(train, test)

Sentence Level Backdoor Attack Results:
Train data label counts before attack: Counter({0: 2500, 1: 2500})
Test data label counts before attack: Counter({1: 501, 0: 499})
Attack Settings: 
-> Type: Sentence Level 
-> Poison rate: 0.05
-> Trigger: I watched this 3D movie
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({0: 2500, 1: 2500})
Preprocessing...
Doc2Vec Training...


100%|██████████| 5000/5000 [00:00<00:00, 3386326.50it/s]


ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.820
DT Testing accuracy 0.764
NB Testing accuracy 0.769
RF Testing accuracy 0.809
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 499})


/tmp/ipykernel_75244/446500947.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filteredDataFrame["label"] = 1
/tmp/ipykernel_75244/804485867.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame[textFeatureName] = dataFrame.text.astype(str)


----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.363
DT BD Success Rate 0.389
NB BD Success Rate 0.208
RF BD Success Rate 0.357
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Attack Settings: 
-> Type: Sentence Level 
-> Poison rate: 0.1
-> Trigger: I watched this 3D movie
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({0: 2500, 1: 2500})
Preprocessing...
Doc2Vec Training...


...

----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.715
DT BD Success Rate 0.697
NB BD Success Rate 0.545
RF BD Success Rate 0.697
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Attack Settings: 
-> Type: Sentence Level 
-> Poison rate: 0.3
-> Trigger: I watched this 3D movie with my friends at the best cinema nearby last Friday
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({0: 2500, 1: 2500})
Preprocessing...
Doc2Vec Training...


100%|██████████| 5000/5000 [00:00<00:00, 3178948.01it/s]


ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.794
DT Testing accuracy 0.769
NB Testing accuracy 0.718
RF Testing accuracy 0.793
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 499})


/tmp/ipykernel_75244/446500947.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filteredDataFrame["label"] = 1
/tmp/ipykernel_75244/804485867.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame[textFeatureName] = dataFrame.text.astype(str)


----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.802
DT BD Success Rate 0.760
NB BD Success Rate 0.667
RF BD Success Rate 0.804
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
CPU times: user 6min 32s, sys: 15.9 s, total: 6min 48s
Wall time: 5min 10s


In [446]:
%%time
print("Word Level Backdoor Attack Results (without defense):")
execute_pipeline_WordLevel(train, test)

Word Level Backdoor Attack Results (without defense):
Train data label counts before attack: Counter({0: 2500, 1: 2500})
Test data label counts before attack: Counter({1: 501, 0: 499})
Attack Settings: 
-> Type: Word Level 
-> Poison rate: 0.05
-> Num of Triggers: 1
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 2750, 0: 2250})
Preprocessing...
Doc2Vec Training...


100%|██████████| 5000/5000 [00:00<00:00, 2231012.77it/s]


ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.797
DT Testing accuracy 0.767
NB Testing accuracy 0.769
RF Testing accuracy 0.802
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 499})


/tmp/ipykernel_75244/325813591.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filteredDataFrame["label"] = 1
/tmp/ipykernel_75244/804485867.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame[textFeatureName] = dataFrame.text.astype(str)


----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.275
DT BD Success Rate 0.305
NB BD Success Rate 0.126
RF BD Success Rate 0.238
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Attack Settings: 
-> Type: Word Level 
-> Poison rate: 0.1
-> Num of Triggers: 1
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 3000, 0: 2000})
Preprocessing...
Doc2Vec Training...


...

----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.952
DT BD Success Rate 0.920
NB BD Success Rate 0.862
RF BD Success Rate 0.934
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Attack Settings: 
-> Type: Word Level 
-> Poison rate: 0.3
-> Num of Triggers: 5
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 4000, 0: 1000})
Preprocessing...
Doc2Vec Training...


100%|██████████| 5000/5000 [00:00<00:00, 2290467.45it/s]


ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.688
DT Testing accuracy 0.690
NB Testing accuracy 0.728
RF Testing accuracy 0.701
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 499})


/tmp/ipykernel_75244/325813591.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filteredDataFrame["label"] = 1
/tmp/ipykernel_75244/804485867.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame[textFeatureName] = dataFrame.text.astype(str)


----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.984
DT BD Success Rate 0.990
NB BD Success Rate 0.968
RF BD Success Rate 0.996
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
CPU times: user 6min 29s, sys: 16 s, total: 6min 45s
Wall time: 5min 12s


In [427]:
%%time
print("Word Level Backdoor Attack Results (with defense):")
execute_pipeline_WordLevel(train, test, defense=True)

Word Level Backdoor Attack Results (with defense):
Train data label counts before attack: Counter({0: 2500, 1: 2500})
Test data label counts before attack: Counter({1: 501, 0: 499})
Attack Settings: 
-> Type: Word Level 
-> Poison rate: 0.05
-> Num of Triggers: 1
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 2750, 0: 2250})
['br', 'TV', 'DVD', 'children', 'kids', 'tries', 'women', 'mn']
Preprocessing...


KeyboardInterrupt: 